In [1]:
import pandas as pd
import warnings
from IPython.utils import io
import sys
import numpy as np
from functools import reduce

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

stars_dir = '~/GitHub/stars-data-builder/'
hos_dir = '~/Desktop/Rush/CMS_HospitalArchives/'

In [2]:
def curate(df):

    try:
        df = df[df['PROVIDER_ID'] != np.nan]
        df['PROVIDER_ID'] = df['PROVIDER_ID'].values.astype(str)
        
        ids = df['PROVIDER_ID'].tolist()
        ids2 = []
        for i in ids:
            if len(i) < 6:
                i = '0' + i
            ids2.append(i)
        df['PROVIDER_ID'] = ids2
        
    except:
        pass
    
    for c in list(df):    
        try:
            df[c] = df[c].str.replace("\t","")
        except:
            pass

    if 'Unnamed: 0' in list(df):
        df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
    return df

## MEASURES USED FOR JULY 2022 File: From PDFs (SAS & CMS)

### Mortality

MORT-30-AMI  
MORT-30-CABG  
MORT-30-COPD  
MORT-30-HF  
MORT-30-PN  
MORT-30-STK  
PSI-4-SURG-COMP  

### Safety of Care

HAI-1  
HAI-2  
HAI-3  
HAI-4  
HAI-5  
HAI-6  
COMP-HIP-KNEE  
PSI-90-Safety  

### Readmission

READM-30-CABG  
READM-30-COPD  
READM-30-Hip-Knee  
READM-30-HOSP-WIDE  
EDAC-30-AMI  
EDAC-30-HF  
EDAC-30-PN  
OP-32  
OP-35 ADM  
OP-35 ED  
OP-36  

### Patient Experience

H-COMP-1  
H-COMP-2  
H-COMP-3  
H-COMP-5  
H-COMP-6  
H-COMP-7  
H-CLEAN-HSP / H-QUIET-HSP  
H-HSP-RATING / H-RECMND  

### Timely and Effective Care

IMM-3  
OP-10  
OP-13  
OP-18b  
OP-2  (<= 100 hospitals reporting)  
OP-22  
OP-23  
OP-29
OP-3b  
OP-8  
PC-01  
SEP-1  


## Retired (2022):
ED-2b  
OP-30 

## Retired in (2023):
OP-33: Percentage of patients receiving appropriate radiation therapy for cancer that has
spread to the bone

## Measure excluded from 2023 due to no more than 100 hospitals reporting performance publicly:
OP-2: Percentage of outpatients with chest pain or possible heart attack who got drugs to
break up blood clots within 30 minutes of arrival

## New measure added for 2023:
HCP COVID-19: COVID-19 Vaccination Coverage Among Health Care Providers


# Use 2022 SAS output files to get the bulk of necessary column labels 

In [3]:
sas_input_df_2022 = pd.read_sas(stars_dir + 'Reproduce_Stars_Input/2022/Input_file/all_data_2022jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")


sas_cols_2022 = list(sas_input_df_2022)

print(len(sas_cols_2022), 'columns in 2022 Stars SAS input file (not all get used by the SAS programs):')
print(sas_cols_2022, '\n')

print('Create columns for 2023 data')
print('Adding HCP_COVID_19 to list of column labels\n')
sas_cols1 = sas_cols_2022 + ['HCP_COVID_19']
print('Removing OP_33 and OP_33_DEN')
sas_cols1.remove('OP_33')
sas_cols1.remove('OP_33_DEN')
sas_cols = list(sas_cols1)

sas_input_df_2022 = curate(sas_input_df_2022)
prvdrs_2022 = sas_input_df_2022['PROVIDER_ID'].unique().tolist()

print(len(prvdrs_2022), 'hospitals in 2022 Stars output file\n')
prvdrs_2022 = sorted(list(set(prvdrs_2022)))

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')


95 columns in 2022 Stars SAS input file (not all get used by the SAS programs):
['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_S

## HAIs

In [4]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Healthcare_Associated_Infections-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['HAI_1_ELIGCASES', 'HAI_1_DOPC', 'HAI_1_SIR', 'HAI_2_ELIGCASES', 'HAI_2_DOPC', 'HAI_2_SIR', 
            'HAI_3_ELIGCASES', 'HAI_3_DOPC', 'HAI_3_SIR', 'HAI_4_ELIGCASES', 'HAI_4_DOPC', 'HAI_4_SIR', 
            'HAI_5_ELIGCASES', 'HAI_5_DOPC', 'HAI_5_SIR', 'HAI_6_ELIGCASES', 'HAI_6_DOPC', 'HAI_6_SIR']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

hai_df = pd.DataFrame(columns=['Facility ID']) 
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    hai_df = hai_df.merge(tdf2, on='Facility ID', how='outer')
    
hai_df.rename(columns={'HAI_1_ELIGCASES': 'HAI_1_DEN_PRED',
                       'HAI_1_DOPC': 'HAI_1_DEN_VOL',
                       'HAI_1_SIR': 'HAI_1',
                       'HAI_2_ELIGCASES': 'HAI_2_DEN_PRED',
                       'HAI_2_DOPC': 'HAI_2_DEN_VOL',
                       'HAI_2_SIR': 'HAI_2',
                       'HAI_3_ELIGCASES': 'HAI_3_DEN_PRED',
                       'HAI_3_DOPC': 'HAI_3_DEN_VOL',
                       'HAI_3_SIR': 'HAI_3',
                       'HAI_4_ELIGCASES': 'HAI_4_DEN_PRED',
                       'HAI_4_DOPC': 'HAI_4_DEN_VOL',
                       'HAI_4_SIR': 'HAI_4',
                       'HAI_5_ELIGCASES': 'HAI_5_DEN_PRED',
                       'HAI_5_DOPC': 'HAI_5_DEN_VOL',
                       'HAI_5_SIR': 'HAI_5',
                       'HAI_6_ELIGCASES': 'HAI_6_DEN_PRED',
                       'HAI_6_DOPC': 'HAI_6_DEN_VOL',
                       'HAI_6_SIR': 'HAI_6',
                       'Facility ID': 'PROVIDER_ID',
                   }, inplace=True)

for c in list(hai_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
hai_df = curate(hai_df)

75 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'EDAC_30_AMI', 'EDAC_30_AMI_DEN', 'EDAC_30_HF', 'EDAC_30_HF_DEN', 'EDAC_30_PN', 'EDAC_30_PN_DEN', 'HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_32', 'OP_32_DEN', 'OP_35_ADM', 'OP_35_ADM_DEN', 'OP_35_ED', 'OP_35_ED_DEN', 'OP_36', 'OP_36_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY',

## Unplanned Hospital Visits


In [5]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Unplanned_Hospital_Visits-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['EDAC_30_AMI', 'EDAC_30_HF', 'EDAC_30_PN', 'OP_32', 'OP_35_ADM', 'OP_35_ED', 'OP_36', 
            'READM_30_CABG', 'READM_30_COPD', 'READM_30_HIP_KNEE', 'READM_30_HOSP_WIDE']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Denominator', 'Measure ID', 'Score'], axis=1)

uhv_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    uhv_df = uhv_df.merge(tdf2, on='Facility ID', how='outer')

uhv_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(uhv_df):
    try:
        sas_cols.remove(c)
    except:
        pass
    
print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
uhv_df = curate(uhv_df)

53 remaining features: ['COMP_HIP_KNEE', 'COMP_HIP_KNEE_DEN', 'HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'MORT_30_AMI', 'MORT_30_AMI_DEN', 'MORT_30_CABG', 'MORT_30_CABG_DEN', 'MORT_30_COPD', 'MORT_30_COPD_DEN', 'MORT_30_HF', 'MORT_30_HF_DEN', 'MORT_30_PN', 'MORT_30_PN_DEN', 'MORT_30_STK', 'MORT_30_STK_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'PSI_90_SAFETY_DEN', 'SEP_1', 'SEP_1_DEN'] 



## COMPLICATIONS AND DEATHS

In [6]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Complications_and_Deaths-Hospital.csv')
#print(df['Measure ID'].unique())

measures = ['MORT_30_AMI', 'MORT_30_CABG', 'MORT_30_COPD', 'MORT_30_HF', 
            'MORT_30_PN', 'MORT_30_STK', 'PSI_04', 'COMP_HIP_KNEE',
            'PSI_90', 
            ]

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Denominator'], axis=1)

cad_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    tdf2[m + '_DEN'] = tdf1['Denominator'].tolist()
    cad_df = cad_df.merge(tdf2, on='Facility ID', how='outer')
    
cad_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'PSI_04': 'PSI_4_SURG_COMP',
                       'PSI_04_DEN': 'PSI_4_SURG_COMP_DEN',
                       'PSI_90': 'PSI_90_SAFETY',
                       'PSI_90_DEN': 'PSI_90_SAFETY_DEN',
                   }, inplace=True)

for c in list(cad_df):
    try:
        sas_cols.remove(c)
    except:
        pass
    
print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
cad_df = curate(cad_df)

35 remaining features: ['HCP_COVID_19', 'H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'IMM_3', 'IMM_3_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_2', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN', 'SEP_1', 'SEP_1_DEN'] 



## TIMELY AND EFFECTIVE CARE

Everything except PC-01, which for 2023 is located in the Maternal Health files of the hospitals data archive

In [7]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Timely_and_Effective_Care-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique(), '\n')

measures = ['IMM_3', 'OP_18b', 'OP_2', 'OP_22', 'OP_23', 'OP_29', 'OP_3b', 'SEP_1', 'HCP_COVID_19']

df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Sample', 'Measure ID', 'Score'], axis=1)

tec_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    if m == 'HCP_COVID_19':
        pass
    else:
        tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    
    tec_df = tec_df.merge(tdf2, on='Facility ID', how='outer')
    
tec_df.rename(columns={'Facility ID': 'PROVIDER_ID', 
                       'OP_3b': 'OP_3B', 
                       'OP_3b_DEN': 'OP_3B_DEN', 
                       'OP_18b': 'OP_18B',
                       'OP_18b_DEN': 'OP_18B_DEN',
                      }, inplace=True)

for c in list(tec_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
tec_df = curate(tec_df)


['EDV' 'ED_2_Strata_1' 'ED_2_Strata_2' 'HCP_COVID_19' 'IMM_3' 'OP_18b'
 'OP_18c' 'OP_2' 'OP_22' 'OP_23' 'OP_29' 'OP_31' 'OP_3b'
 'SAFE_USE_OF_OPIOIDS' 'SEP_1' 'SEP_SH_3HR' 'SEP_SH_6HR' 'SEV_SEP_3HR'
 'SEV_SEP_6HR' 'STK_02' 'STK_03' 'STK_05' 'STK_06' 'VTE_1' 'VTE_2'] 

18 remaining features: ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN', 'PC_01', 'PC_01_DEN'] 



## TIMELY AND EFFECTIVE CARE

For 2023, PC-01 is located in the Maternal Health files of the hospitals data archive

In [8]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Maternal_Health-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['PC_01']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score', 'Sample'], axis=1)

tec2_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    tdf2[m + '_DEN'] = tdf1['Sample'].tolist()
    tec2_df = tec2_df.merge(tdf2, on='Facility ID', how='outer')
    
tec2_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)

for c in list(tec2_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')

tec2_df = curate(tec2_df)


['PC_01' 'PC_05' 'SM_7']
16 remaining features: ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_GLOB_STAR_RATING', 'H_INDI_STAR_RATING', 'H_NUMB_COMP', 'H_RESP_RATE_P', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



## HCAHPS

In [9]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/HCAHPS-Hospital.csv')
print(df['HCAHPS Measure ID'].unique())

measures = ['H_COMP_1_STAR_RATING', 'H_COMP_2_STAR_RATING', 'H_COMP_3_STAR_RATING', 'H_COMP_5_STAR_RATING', 
            'H_COMP_6_STAR_RATING', 'H_COMP_7_STAR_RATING', 'H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 
            'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING']

df = df[df['HCAHPS Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'HCAHPS Measure ID', 'Patient Survey Star Rating', 
                        'Number of Completed Surveys', 'Survey Response Rate Percent'], axis=1)

HCAHPS_df = pd.DataFrame(columns=['Facility ID'])
for i, m in enumerate(measures):
    tdf1 = df[df['HCAHPS Measure ID'] == m]
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Patient Survey Star Rating'].tolist()
    if i == 0:
        tdf2['H_NUMB_COMP'] = tdf1['Number of Completed Surveys'].tolist()
        tdf2['H_RESP_RATE_P'] = tdf1['Survey Response Rate Percent'].tolist()
        
    HCAHPS_df = HCAHPS_df.merge(tdf2, on='Facility ID', how='outer')
    

HCAHPS_df.rename(columns={'Facility ID': 'PROVIDER_ID'}, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_HSP_RATING_STAR_RATING'] = HCAHPS_df['H_HSP_RATING_STAR_RATING'].astype(int)
HCAHPS_df['H_RECMND_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_RECMND_STAR_RATING'] = HCAHPS_df['H_RECMND_STAR_RATING'].astype(int)
HCAHPS_df['H_GLOB_STAR_RATING'] = np.round((HCAHPS_df['H_HSP_RATING_STAR_RATING'] + HCAHPS_df['H_RECMND_STAR_RATING']) / 2, 1)
HCAHPS_df['H_GLOB_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df['H_CLEAN_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_CLEAN_STAR_RATING'] = HCAHPS_df['H_CLEAN_STAR_RATING'].astype(int)
HCAHPS_df['H_QUIET_STAR_RATING'].replace('Not Available', 0, inplace=True)
HCAHPS_df['H_QUIET_STAR_RATING'] = HCAHPS_df['H_QUIET_STAR_RATING'].astype(int)
HCAHPS_df['H_INDI_STAR_RATING'] = np.round((HCAHPS_df['H_CLEAN_STAR_RATING'] + HCAHPS_df['H_QUIET_STAR_RATING']) / 2, 1)
HCAHPS_df['H_INDI_STAR_RATING'].replace(0, np.nan, inplace=True)

HCAHPS_df.drop(labels = ['H_CLEAN_STAR_RATING',  'H_QUIET_STAR_RATING', 'H_RECMND_STAR_RATING', 'H_HSP_RATING_STAR_RATING'], axis=1, inplace=True)

for c in list(HCAHPS_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
HCAHPS_df = curate(HCAHPS_df)

['H_COMP_1_A_P' 'H_COMP_1_SN_P' 'H_COMP_1_U_P' 'H_COMP_1_LINEAR_SCORE'
 'H_COMP_1_STAR_RATING' 'H_NURSE_RESPECT_A_P' 'H_NURSE_RESPECT_SN_P'
 'H_NURSE_RESPECT_U_P' 'H_NURSE_LISTEN_A_P' 'H_NURSE_LISTEN_SN_P'
 'H_NURSE_LISTEN_U_P' 'H_NURSE_EXPLAIN_A_P' 'H_NURSE_EXPLAIN_SN_P'
 'H_NURSE_EXPLAIN_U_P' 'H_COMP_2_A_P' 'H_COMP_2_SN_P' 'H_COMP_2_U_P'
 'H_COMP_2_LINEAR_SCORE' 'H_COMP_2_STAR_RATING' 'H_DOCTOR_RESPECT_A_P'
 'H_DOCTOR_RESPECT_SN_P' 'H_DOCTOR_RESPECT_U_P' 'H_DOCTOR_LISTEN_A_P'
 'H_DOCTOR_LISTEN_SN_P' 'H_DOCTOR_LISTEN_U_P' 'H_DOCTOR_EXPLAIN_A_P'
 'H_DOCTOR_EXPLAIN_SN_P' 'H_DOCTOR_EXPLAIN_U_P' 'H_COMP_3_A_P'
 'H_COMP_3_SN_P' 'H_COMP_3_U_P' 'H_COMP_3_LINEAR_SCORE'
 'H_COMP_3_STAR_RATING' 'H_CALL_BUTTON_A_P' 'H_CALL_BUTTON_SN_P'
 'H_CALL_BUTTON_U_P' 'H_BATH_HELP_A_P' 'H_BATH_HELP_SN_P'
 'H_BATH_HELP_U_P' 'H_COMP_5_A_P' 'H_COMP_5_SN_P' 'H_COMP_5_U_P'
 'H_COMP_5_LINEAR_SCORE' 'H_COMP_5_STAR_RATING' 'H_MED_FOR_A_P'
 'H_MED_FOR_SN_P' 'H_MED_FOR_U_P' 'H_SIDE_EFFECTS_A_P'
 'H_SIDE_EFFECTS_SN_P'

## Outpatient Imaging Efficiency

The July 2023 Overall Star Ratings were calculated using the measure data from the January 2023 Care Compare update, with the re-released OP-13 measure data publicly reported in April 2023 on Care Compare.

Make two dataframes: 1 for OP-13 (April 2023) and 1 for other measures (Jan 2023)

In [10]:
df = pd.read_csv(hos_dir + '2023/hospitals_04_2023/Outpatient_Imaging_Efficiency-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['OP-13']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-13': 'OP_13',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
oie_df1 = curate(oie_df)


['OP-10' 'OP-13' 'OP-39' 'OP-8']
5 remaining features: ['OP_10', 'OP_10_DEN', 'OP_13_DEN', 'OP_8', 'OP_8_DEN'] 



In [11]:
df = pd.read_csv(hos_dir + '2023/hospitals_01_2023/Outpatient_Imaging_Efficiency-Hospital.csv', encoding='latin-1')
print(df['Measure ID'].unique())

measures = ['OP-8', 'OP-10']
df = df[df['Measure ID'].isin(measures)]
df = df.filter(items = ['Facility ID', 'Measure ID', 'Score'], axis=1)

oie_df = pd.DataFrame(columns=['Facility ID'])
for m in measures:
    tdf1 = df[df['Measure ID'] == m]
    
    tdf2 = pd.DataFrame(columns=['Facility ID', m]) 
    tdf2['Facility ID'] = tdf1['Facility ID'].tolist()
    tdf2[m] = tdf1['Score'].tolist()
    
    oie_df = oie_df.merge(tdf2, on='Facility ID', how='outer')
    
oie_df.rename(columns={'Facility ID': 'PROVIDER_ID',
                       'OP-8': 'OP_8',
                       'OP-10': 'OP_10',
                       'OP-13': 'OP_13',
                   }, inplace=True)

for c in list(oie_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')
oie_df2 = curate(oie_df)


['OP-10' 'OP-13' 'OP-39' 'OP-8']
3 remaining features: ['OP_10_DEN', 'OP_13_DEN', 'OP_8_DEN'] 



## MERGE DATAFRAMES

In [12]:
main_df = tec_df.merge(tec2_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(cad_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(HCAHPS_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(uhv_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(hai_df, on='PROVIDER_ID', how='outer')
main_df = main_df.merge(oie_df2, on='PROVIDER_ID', how='outer')

prvdrs = main_df['PROVIDER_ID'].unique().tolist()
oie_df1 = oie_df1[oie_df1['PROVIDER_ID'].isin(prvdrs)]
main_df = main_df.merge(oie_df1, on='PROVIDER_ID', how='outer')

main_df['OP_10_DEN'] = np.nan
main_df['OP_13_DEN'] = np.nan
main_df['OP_8_DEN'] = np.nan
main_df['HCP_COVID_19_DEN'] = np.nan

for c in list(main_df):
    try:
        sas_cols.remove(c)
    except:
        pass

print(len(sas_cols), 'remaining features:', sorted(sas_cols), '\n')

ls = np.setdiff1d(list(main_df), sas_cols1)
main_df = main_df.filter(items=sas_cols1, axis=1)
print('main_df.shape:', main_df.shape)

for col in list(main_df):
    if col != 'PROVIDER_ID':
        main_df[col] = pd.to_numeric(main_df[col], errors='coerce')


0 remaining features: [] 

main_df.shape: (4848, 94)


## Multiple select features by 0.01

Several features in the SAS file are represented as rates on the scale of 0 to 1, but are represented in the hospitals data archive as percentages on the scale of 0 to 100.

In [13]:
ls = list(main_df)
ls.remove('PROVIDER_ID')

prvdrs_main1 = sorted(main_df['PROVIDER_ID'].tolist())

ls = ['READM_30_HIP_KNEE', 'READM_30_COPD', 'MORT_30_STK', 'MORT_30_PN',
      'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'OP_22',
      'OP_23', 'OP_29', 'IMM_3', 'PC_01', 'SEP_1', 'MORT_30_CABG',
      'READM_30_CABG', 'READM_30_HOSP_WIDE', 'OP_2', 'OP_8',
      'OP_10', 'OP_13', 'HCP_COVID_19']

for l in ls: 
    main_df[l] = main_df[l] * 0.01


## Potential source of discrepancy #1:

The "Overall Hospital Quality Star Rating on Care Compare: July 2023 Updates and Specifications Report" states the following:

"Note: The total number of hospitals in the Care Compare dataset as of January 2023 is **4,654 hospitals**."  
However, the derived dataset from hospitals data archives has **4,856 hospitals**.

The report also states: "Results shown are for all hospitals meeting the reporting criteria (N = 3076). Please note results included in this report may differ from the results posted on Care Compare due to data suppressed by CMS for one or more quarters. CMS may suppress data for various reasons, like data inaccuracies."



## Prepare data for import into SAS as csv

**Issue:** Python can read SAS files but it can't write them. Only SAS can write SAS files. Consequently, the dataframe produced here must be exported as a CSV file. When importing the CSV into SAS, SAS has a problem interpreting the CMS IDs of hospitals because it tries to interpret some IDs as numbers and others as strings (character variables). This is because VHA hospitals contain an 'F' at the end of their IDs.

**Solution:** Remove the 'F' suffix from VHA hospitals and replaced it with 666666. Doing so alleviate the issue of importing the CSV file and not causing in error in reading CMS IDs, while preventing ID conflicts.

In [14]:
prvdrs = []
for p in main_df['PROVIDER_ID'].tolist():
    if 'F' in p:
        p = p[:-1]
        p = p + '666666'
    prvdrs.append(p)

main_df['PROVIDER_ID'] = prvdrs

print(list(main_df))
main_df.head()

main_df['PROVIDER_ID'] = pd.to_numeric(main_df['PROVIDER_ID'], errors='coerce')
main_df.sort_values(by=['PROVIDER_ID'], ascending = True, inplace = True)
#main_df.to_csv(stars_dir + "Reproduce_Stars_Input/2023/Input_File/all_data_2023jul.csv", index=False)


['PROVIDER_ID', 'HAI_1_DEN_VOL', 'HAI_2_DEN_VOL', 'HAI_3_DEN_VOL', 'HAI_4_DEN_VOL', 'HAI_5_DEN_VOL', 'HAI_6_DEN_VOL', 'HAI_1_DEN_PRED', 'HAI_2_DEN_PRED', 'HAI_3_DEN_PRED', 'HAI_4_DEN_PRED', 'HAI_5_DEN_PRED', 'HAI_6_DEN_PRED', 'HAI_1', 'HAI_2', 'HAI_3', 'HAI_4', 'HAI_5', 'HAI_6', 'READM_30_HOSP_WIDE', 'READM_30_HIP_KNEE', 'EDAC_30_HF', 'READM_30_COPD', 'EDAC_30_AMI', 'EDAC_30_PN', 'MORT_30_STK', 'MORT_30_PN', 'MORT_30_HF', 'MORT_30_COPD', 'MORT_30_AMI', 'COMP_HIP_KNEE', 'READM_30_HOSP_WIDE_DEN', 'READM_30_HIP_KNEE_DEN', 'EDAC_30_HF_DEN', 'READM_30_COPD_DEN', 'EDAC_30_AMI_DEN', 'EDAC_30_PN_DEN', 'MORT_30_STK_DEN', 'MORT_30_PN_DEN', 'MORT_30_HF_DEN', 'MORT_30_COPD_DEN', 'MORT_30_AMI_DEN', 'COMP_HIP_KNEE_DEN', 'OP_2', 'OP_2_DEN', 'OP_3B', 'OP_3B_DEN', 'OP_8', 'OP_8_DEN', 'OP_10', 'OP_10_DEN', 'OP_13', 'OP_13_DEN', 'OP_18B', 'OP_18B_DEN', 'OP_22', 'OP_22_DEN', 'OP_23', 'OP_23_DEN', 'OP_29', 'OP_29_DEN', 'PSI_4_SURG_COMP', 'PSI_4_SURG_COMP_DEN', 'PSI_90_SAFETY', 'IMM_3_DEN', 'IMM_3', 'PC_01'

#    

-------

## Post-hoc comparison between the constructed data file (used as input data for stars programs) and the actual public released data file

In [15]:
tdf = main_df.copy(deep=True)

## Replace the imputed 666666 suffixes of VHA hospitals with their original 'F' suffix
prvdrs1 = []
for p in tdf['PROVIDER_ID'].tolist():
    p = str(p)
    if '666666' in p:
        p = p[:-6]
        p = p + 'F'
    while len(p) < 6:
        p = '0' + p
    prvdrs1.append(p)
tdf['PROVIDER_ID'] = prvdrs1


Import publicly released SAS database file (alldata_2023jul.sas7bdat)

In [16]:
sas_input_df = pd.read_sas(stars_dir + 'Reproduce_Stars_Input/2023/Input_File/alldata_2023jul.sas7bdat', 
                           format = 'sas7bdat', encoding = "utf8")


sas_input_df = curate(sas_input_df)
sas_cols = list(sas_input_df)
remaining_ls = list(sas_cols)
print('sas_input_df.shape:', sas_input_df.shape)

labels = list(sas_input_df)
labels.remove('PROVIDER_ID')
print(len(labels), 'individual measures in the publicly-released SAS dataframe\n')
del labels

sas_input_df.sort_values(by=['PROVIDER_ID'], ascending=True, inplace=True)
sas_input_df.head()


sas_input_df.shape: (4654, 95)
94 individual measures in the publicly-released SAS dataframe



,PROVIDER_ID,HAI_1_DEN_VOL,HAI_2_DEN_VOL,HAI_3_DEN_VOL,HAI_4_DEN_VOL,HAI_5_DEN_VOL,HAI_6_DEN_VOL,HAI_1_DEN_PRED,HAI_2_DEN_PRED,HAI_3_DEN_PRED,HAI_4_DEN_PRED,HAI_5_DEN_PRED,HAI_6_DEN_PRED,HAI_1,HAI_2,HAI_3,HAI_4,HAI_5,HAI_6,READM_30_HOSP_WIDE,READM_30_HIP_KNEE,EDAC_30_HF,READM_30_COPD,EDAC_30_AMI,EDAC_30_PN,MORT_30_STK,MORT_30_PN,MORT_30_HF,MORT_30_COPD,MORT_30_AMI,COMP_HIP_KNEE,READM_30_HOSP_WIDE_DEN,READM_30_HIP_KNEE_DEN,EDAC_30_HF_DEN,READM_30_COPD_DEN,EDAC_30_AMI_DEN,EDAC_30_PN_DEN,MORT_30_STK_DEN,MORT_30_PN_DEN,MORT_30_HF_DEN,MORT_30_COPD_DEN,MORT_30_AMI_DEN,COMP_HIP_KNEE_DEN,OP_2,OP_2_DEN,OP_3B,OP_3B_DEN,OP_8,OP_8_DEN,OP_10,OP_10_DEN,OP_13,OP_13_DEN,OP_18B,OP_18B_DEN,OP_22,OP_22_DEN,OP_23,OP_23_DEN,OP_29,OP_29_DEN,PSI_4_SURG_COMP,PSI_4_SURG_COMP_DEN,PSI_90_SAFETY,IMM_3_DEN,IMM_3,HCP_COVID_19_DEN,HCP_COVID_19,PC_01,PC_01_DEN,SEP_1,SEP_1_DEN,H_RESP_RATE_P,H_COMP_1_STAR_RATING,H_COMP_2_STAR_RATING,H_COMP_3_STAR_RATING,H_COMP_5_STAR_RATING,H_COMP_6_STAR_RATING,H_COMP_7_STAR_RATING,H_GLOB_STAR_RATING,H_INDI_STAR_RATING,H_NUMB_COMP,PSI_90_SAFETY_DEN,MORT_30_CABG,MORT_30_CABG_DEN,READM_30_CABG,READM_30_CABG_DEN,OP_32,OP_32_DEN,OP_35_ADM,OP_35_ADM_DEN,OP_35_ED,OP_35_ED_DEN,OP_36,OP_36_DEN
0,010001,10024.0,17731.0,154.0,200.0,101908.0,101451.0,10.597,26.630,4.548,1.845,9.412,72.686,0.661,0.300,1.099,0.0,0.850,0.660,0.142,0.042,21.8,0.199,1.9,-1.5,0.164,0.159,0.083,0.085,0.124,0.024,3058.0,98.0,755.0,202.0,319.0,436.0,489.0,407.0,630.0,182.0,317.0,102.0,NaN,NaN,NaN,NaN,0.425,146.0,0.057,1488.0,0.067,208.0,205.0,323.0,0.03,51079.0,NaN,NaN,0.81,16.0,173.39,120.0,1.01,3795.0,0.97,2323.0,0.737,0.09,34.0,0.46,146.0,15.0,2.0,3.0,2.0,4.0,4.0,3.0,3.0,3.5,434.0,2046.895485,0.047,172.0,0.117,165.0,14.1,254.0,10.2,214.0,4.7,214.0,1.0,688.0
1,010005,3713.0,8670.0,88.0,NaN,38413.0,35686.0,2.450,4.995,2.512,NaN,1.999,10.484,3.673,1.201,1.194,NaN,0.000,0.858,0.139,0.040,9.3,0.176,4.7,2.2,0.166,0.218,0.169,0.081,0.126,0.018,1258.0,178.0,157.0,234.0,38.0,361.0,100.0,369.0,153.0,195.0,52.0,134.0,NaN,NaN,NaN,NaN,0.545,191.0,0.138,1214.0,0.043,208.0,146.0,1003.0,0.03,54503.0,0.73,15.0,0.99,108.0,142.88,35.0,0.91,2593.0,0.90,2026.0,0.821,0.01,194.0,0.59,242.0,16.0,3.0,4.0,2.0,3.0,3.0,3.0,3.0,3.5,717.0,819.043002,NaN,NaN,NaN,NaN,14.6,850.0,11.1,108.0,5.7,108.0,0.9,362.0
2,010006,7318.0,11755.0,91.0,NaN,62709.0,54159.0,7.924,15.296,2.523,NaN,4.164,22.618,0.757,0.196,0.396,NaN,1.441,0.088,0.142,0.048,-2.3,0.177,25.9,42.3,0.189,0.178,0.122,0.078,0.165,0.034,2555.0,246.0,550.0,235.0,312.0,538.0,261.0,528.0,468.0,209.0,295.0,234.0,NaN,NaN,NaN,NaN,0.412,97.0,0.110,1168.0,0.014,217.0,144.0,363.0,0.01,41137.0,0.57,14.0,0.88,75.0,157.42,84.0,1.10,2292.0,0.64,2694.0,0.651,0.00,37.0,0.58,142.0,17.0,2.0,3.0,1.0,2.0,3.0,2.0,2.0,2.5,1358.0,1487.163359,0.035,117.0,0.156,109.0,12.5,1505.0,NaN,NaN,NaN,NaN,1.1,468.0
3,010007,NaN,NaN,NaN,NaN,NaN,5413.0,NaN,NaN,NaN,NaN,NaN,2.148,NaN,NaN,NaN,NaN,NaN,0.466,0.151,NaN,36.3,0.197,NaN,-12.6,NaN,0.217,0.139,0.103,NaN,NaN,272.0,NaN,51.0,72.0,NaN,99.0,NaN,106.0,45.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059,169.0,NaN,NaN,119.0,1202.0,0.03,11120.0,NaN,NaN,0.63,68.0,NaN,NaN,0.99,318.0,0.61,277.0,0.574,NaN,NaN,0.93,55.0,23.0,3.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,173.0,142.073902,NaN,NaN,NaN,NaN,15.3,118.0,NaN,NaN,NaN,NaN,1.0,56.0
4,010008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.145,NaN,NaN,NaN,NaN,NaN,NaN,0.197,NaN,NaN,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021,97.0,NaN,NaN,113.0,346.0,0.00,6205.0,NaN,NaN,0.52,23.0,NaN,NaN,NaN,125.0,0.46,169.0,0.623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.3,62.0,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
prvdrs_sas = sorted(sas_input_df['PROVIDER_ID'].tolist())
prvdrs_main1 = sorted(tdf['PROVIDER_ID'].tolist())

ps = []
for p in prvdrs_sas:
    if p not in prvdrs_main1:
        ps.append(p)

print(len(ps), 'hospitals in the SAS file that are not in the reconstructed file:')
print(ps, '\n')


ps = []
for p in prvdrs_main1:
    if p not in prvdrs_sas:
        ps.append(p)

print(len(ps), 'hospitals in the reconstructed file that are not in the SAS file:')
print(ps, '\n')


1 hospitals in the SAS file that are not in the reconstructed file:
['670265'] 

195 hospitals in the reconstructed file that are not in the SAS file:
['013300', '013301', '02013F', '02014F', '031307', '033302', '043300', '043301', '05015F', '05020F', '05022F', '05039F', '05041F', '050546', '050849', '051309', '053300', '053301', '053302', '053303', '053304', '053305', '053306', '053308', '053309', '053311', '06003F', '061323', '063301', '063303', '070038', '070040', '073300', '083300', '093300', '10013F', '10021F', '100298', '103300', '103301', '103304', '11032F', '11033F', '11035F', '113300', '113301', '12001F', '123300', '123301', '143300', '143301', '143302', '150193', '161315', '17002F', '173300', '18003F', '190300', '190302', '190315', '19050F', '191308', '191314', '191319', '193300', '193301', '21007F', '21010F', '213300', '213301', '223300', '223302', '223303', '223304', '233300', '243300', '243302', '25039F', '251307', '251314', '251339', '26002F', '261315', '263301', '263302'

Filter the reconstructed data and the publicly released data so they have a common set of providers


In [18]:
prvdrs = tdf['PROVIDER_ID'].unique()
sas_tdf = sas_input_df[sas_input_df['PROVIDER_ID'].isin(prvdrs)]

prvdrs = sas_tdf['PROVIDER_ID'].unique()
tdf = tdf[tdf['PROVIDER_ID'].isin(prvdrs)]

print('tdf.shape:', tdf.shape)
print('sas_tdf.shape:', sas_tdf.shape)

tdf.sort_values(by=['PROVIDER_ID'], inplace=True)
sas_tdf.sort_values(by=['PROVIDER_ID'], inplace=True)

tdf.fillna(0, inplace=True)
sas_tdf.fillna(0, inplace=True)

tdf.shape: (4653, 94)
sas_tdf.shape: (4653, 95)


In [19]:
def get_apd(obs, exp, label=None):
    # a function to calculate average percent difference
    
    if len(obs) != len(exp):
        raise ValueError("Both lists must have the same length")
    
    if min(obs) < 0 or min(exp) < 0:
        if min(obs) < 0:
            #print('Error -- ', label, ': min(obs) = ', min(obs))
            obs = -1 *np.array(obs)
            obs = obs.tolist()
        if min(exp) < 0:
            #print('Error -- ', label, ': min(exp) = ', min(exp))
            exp = -1 *np.array(exp)
            exp = exp.tolist()
            
    n = len(obs)
    apd_values = []

    for i in range(n):
        numerator = abs(obs[i] - exp[i])
        denominator = (obs[i] + exp[i]) / 2

        # Avoid division by zero
        if denominator == 0:
            apd = 0  # Both obs[i] and exp[i] are zero, no error
        else:
            apd = numerator / denominator * 100

        apd_values.append(apd)

    average_apd = sum(apd_values) / n
    return average_apd


In [20]:
apd_ls = []
n_perfect = 0
n_miss = 0
n_tot = 0

labels = ['MORT_30_AMI',
          'MORT_30_CABG',
          'MORT_30_COPD', 
          'MORT_30_HF',
          'MORT_30_PN',
          'MORT_30_STK',
          'PSI_4_SURG_COMP',
          'COMP_HIP_KNEE', 
          'HAI_1',
          'HAI_2',
          'HAI_3',
          'HAI_4',
          'HAI_5',
          'HAI_6',
          'PSI_90_SAFETY',
          'EDAC_30_AMI',
          'EDAC_30_HF',
          'EDAC_30_PN', 
          'OP_32',
          'READM_30_CABG',
          'READM_30_COPD',
          'READM_30_HIP_KNEE',
          'READM_30_HOSP_WIDE',
          'OP_35_ADM', 
          'OP_35_ED',
          'OP_36',
          'H_COMP_1_STAR_RATING',
          'H_COMP_2_STAR_RATING',
          'H_COMP_3_STAR_RATING',
          'H_COMP_5_STAR_RATING',
          'H_COMP_6_STAR_RATING',
          'H_COMP_7_STAR_RATING',
          'H_GLOB_STAR_RATING',
          'H_INDI_STAR_RATING',
          'IMM_3',
          'OP_22',
          'OP_23',
          'OP_29',
          #'OP_30',
          #'OP_33',
          'PC_01',
          'SEP_1',
          'OP_3B',
          'OP_18B',
          #'ED_2B',
          'OP_8',
          'OP_10',
          'OP_13',
          #'OP_2', # was excluded from 2023 for having too few hospitals (<= 100) reporting
          'HCP_COVID_19',
         ]

print(len(labels), 'individual measures used in stars SAS files')
for label in labels:
        
    obs = tdf[label].astype('float').tolist()
    exp = sas_tdf[label].astype('float').tolist()
    
    result = get_apd(obs, exp, label)
    apd_ls.append(result)
    
    #if result < 0 or result > 1:
    print(label, '| APD:', round(result, 4))
    
    for i, o in enumerate(obs):
        e = exp[i]
        if o == e:
            n_perfect += 1
            n_tot += 1
        else:
            n_miss += 1
            n_tot += 1

print('\n')
print('Results for Average Percent Difference:')
print('median:', np.nanmedian(apd_ls))
q1 = np.percentile(apd_ls, 25)
q3 = np.percentile(apd_ls, 75)
print('Q1:', q1)
print('Q3:', q3)

print('% of all measures that were perfectly reproduced:', 100*n_perfect/n_tot)

46 individual measures used in stars SAS files
MORT_30_AMI | APD: 0.0
MORT_30_CABG | APD: 0.0
MORT_30_COPD | APD: 0.1719
MORT_30_HF | APD: 0.2579
MORT_30_PN | APD: 0.2579
MORT_30_STK | APD: 0.1289
PSI_4_SURG_COMP | APD: 0.0
COMP_HIP_KNEE | APD: 0.1719
HAI_1 | APD: 0.0
HAI_2 | APD: 0.0
HAI_3 | APD: 0.0
HAI_4 | APD: 0.0
HAI_5 | APD: 0.0
HAI_6 | APD: 0.086
PSI_90_SAFETY | APD: 0.0
EDAC_30_AMI | APD: 0.0
EDAC_30_HF | APD: 0.1289
EDAC_30_PN | APD: 0.1289
OP_32 | APD: 0.1289
READM_30_CABG | APD: 0.0
READM_30_COPD | APD: 0.1719
READM_30_HIP_KNEE | APD: 0.1719
READM_30_HOSP_WIDE | APD: 0.2579
OP_35_ADM | APD: 0.0
OP_35_ED | APD: 0.0
OP_36 | APD: 0.0
H_COMP_1_STAR_RATING | APD: 0.043
H_COMP_2_STAR_RATING | APD: 0.043
H_COMP_3_STAR_RATING | APD: 0.043
H_COMP_5_STAR_RATING | APD: 0.043
H_COMP_6_STAR_RATING | APD: 0.043
H_COMP_7_STAR_RATING | APD: 0.043
H_GLOB_STAR_RATING | APD: 0.043
H_INDI_STAR_RATING | APD: 0.043
IMM_3 | APD: 5.6738
OP_22 | APD: 4.1264
OP_23 | APD: 0.1289
OP_29 | APD: 4.3413
PC

#        

----

## Load file of predicted star ratings and scores

This file results from running the above file through the modified 2022 stars SAS pack.

In [21]:
path = stars_dir + '2023/2023-07 Stars Predictions/SAS_output/CMS_Stars_Jul_2023_Exp_Check.csv'
df_pred = pd.read_csv(path)

df_pred = df_pred[~df_pred['star'].isin([np.nan, float("NaN")])]
print(df_pred.shape)
df_pred.head()

(3073, 27)


,PROVIDER_ID,Std_Outcomes_Mortality_score,Std_Outcomes_Readmission_score,Std_Outcomes_Safety_score,Std_PatientExp_score,Std_Process_score,std_weight_PatientExperience,std_weight_Readmission,std_weight_Mortality,std_weight_safety,std_weight_Process,weight_PatientExperience,weight_Outcomes_Readmission,weight_Outcomes_Mortality,weight_Outcomes_Safety,weight_Process,summary_score,Outcomes_Mortality_cnt,Outcomes_safety_cnt,Outcomes_Readmission_cnt,Patient_Experience_cnt,Process_cnt,Total_measure_group_cnt,MortSafe_Group_cnt,report_indicator,cnt_grp,star
0,10001,-0.648411,0.284587,0.310804,-0.138318,-1.039587,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.166845,7,8,11,8,10,5,2,1,3) # of groups=5,3.0
1,10005,-1.566438,0.560509,-0.228638,-0.176647,-0.315833,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.348367,6,7,10,8,11,5,2,1,3) # of groups=5,2.0
2,10006,-1.696213,-0.554222,-0.087562,-1.248599,-0.188854,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.811713,7,7,9,8,11,5,2,1,3) # of groups=5,1.0
3,10007,-2.409810,-0.487315,0.027365,0.981590,-0.655021,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.494000,3,2,6,8,7,4,1,1,2) # of groups=4,2.0
5,10011,-0.518784,-0.622974,0.426891,0.123447,-2.399212,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.418018,7,7,9,8,8,5,2,1,3) # of groups=5,2.0


In [22]:
ls = [250, 668, 872, 803, 483]
n = 0
for s in [1,2,3,4,5]:
    tdf = df_pred[df_pred['star'] == s]
    print('predicted no. of ' + str(s) + ' star hospitals:', tdf.shape[0])
    print('actual no. of ' + str(s) + ' star hospitals:', ls[s-1], '\n')
    n += tdf.shape[0]
    
print(n)
print(sum(ls))

predicted no. of 1 star hospitals: 271
actual no. of 1 star hospitals: 250 

predicted no. of 2 star hospitals: 704
actual no. of 2 star hospitals: 668 

predicted no. of 3 star hospitals: 865
actual no. of 3 star hospitals: 872 

predicted no. of 4 star hospitals: 804
actual no. of 4 star hospitals: 803 

predicted no. of 5 star hospitals: 429
actual no. of 5 star hospitals: 483 

3073
3076


## Load file of actual star ratings and scores


In [23]:
path = '~/Desktop/Rush/CMS_Stars/2023/2023-07 Stars Release/SAS_CSV_output/CMS_Stars_Jul_2023.csv'
df_actual = pd.read_csv(path)

df_actual = df_actual[~df_actual['star'].isin([np.nan, float("NaN")])]
print(df_actual.shape)
df_actual.head()

(3076, 27)


,PROVIDER_ID,Std_Outcomes_Mortality_score,Std_Outcomes_Readmission_score,Std_Outcomes_Safety_score,Std_PatientExp_score,Std_Process_score,std_weight_PatientExperience,std_weight_Readmission,std_weight_Mortality,std_weight_safety,std_weight_Process,weight_PatientExperience,weight_Outcomes_Readmission,weight_Outcomes_Mortality,weight_Outcomes_Safety,weight_Process,summary_score,Outcomes_Mortality_cnt,Outcomes_safety_cnt,Outcomes_Readmission_cnt,Patient_Experience_cnt,Process_cnt,Total_measure_group_cnt,MortSafe_Group_cnt,report_indicator,cnt_grp,star
0,010001,-0.647058,0.284081,0.312074,-0.127836,-1.024044,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.162208,7,8,11,8,10,5,2,1,3) # of groups=5,3.0
1,010005,-1.564103,0.560369,-0.237844,-0.166838,-0.302742,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.346181,6,7,10,8,11,5,2,1,3) # of groups=5,2.0
2,010006,-1.694318,-0.554988,-0.089526,-1.241108,-0.179350,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.809109,7,7,9,8,11,5,2,1,3) # of groups=5,1.0
3,010007,-2.407150,-0.488553,0.022657,0.993806,-0.659760,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.492604,3,2,6,8,7,4,1,1,2) # of groups=4,2.0
5,010011,-0.517349,-0.624302,0.428770,0.134223,-2.385055,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.413511,7,7,9,8,8,5,2,1,3) # of groups=5,2.0


In [24]:
ls = [250, 668, 872, 803, 483]
n = 0
for s in [1,2,3,4,5]:
    tdf = df_actual[df_actual['star'] == s]
    print('predicted no. of ' + str(s) + ' star hospitals:', tdf.shape[0])
    print('actual no. of ' + str(s) + ' star hospitals:', ls[s-1], '\n')
    n += tdf.shape[0]
    
print(n)
print(sum(ls))

predicted no. of 1 star hospitals: 250
actual no. of 1 star hospitals: 250 

predicted no. of 2 star hospitals: 668
actual no. of 2 star hospitals: 668 

predicted no. of 3 star hospitals: 872
actual no. of 3 star hospitals: 872 

predicted no. of 4 star hospitals: 803
actual no. of 4 star hospitals: 803 

predicted no. of 5 star hospitals: 483
actual no. of 5 star hospitals: 483 

3076
3076


In [25]:
## Replace the imputed 666666 suffixes of VHA hospitals with their original 'F' suffix
prvdrs1 = []
for p in df_pred['PROVIDER_ID'].tolist():
    p = str(p)
    if '666666' in p:
        p = p[:-6]
        p = p + 'F'
    while len(p) < 6:
        p = '0' + p
    prvdrs1.append(p)
    
df_pred['PROVIDER_ID'] = prvdrs1

prvdrs2 = df_actual['PROVIDER_ID'].unique()

# Get providers in prvdrs1 that are NOT in prvdrs2
not_in_ls1 = np.setdiff1d(prvdrs1, prvdrs2)
print(len(not_in_ls1))

# Get providers in prvdrs2 that are NOT in prvdrs1
not_in_ls2 = np.setdiff1d(prvdrs2, prvdrs1)
print(len(not_in_ls2))

print(not_in_ls1, '\n')
print(not_in_ls2)

5
8
['12001F' '21007F' '45068F' '45069F' '50005F'] 

['041323' '11031F' '14004F' '36008F' '38004F' '381320' '501332' '521325']


In [26]:

prvdrs = df_actual['PROVIDER_ID'].unique()

df_pred = df_pred[df_pred['PROVIDER_ID'].isin(prvdrs)]

prvdrs = df_pred['PROVIDER_ID'].unique()
df_actual = df_actual[df_actual['PROVIDER_ID'].isin(prvdrs)]

df_pred.sort_values(by=['PROVIDER_ID'], inplace=True)
df_actual.sort_values(by=['PROVIDER_ID'], inplace=True)

if df_pred['PROVIDER_ID'].tolist() == df_actual['PROVIDER_ID'].tolist():
    print('The ordered list of providers in pred and actual are the same')
else:
    print('The ordered list of providers in pred and actual are NOT the same')

print('df_actual.shape:', df_actual.shape)
print('df_pred.shape:', df_pred.shape)

The ordered list of providers in pred and actual are the same
df_actual.shape: (3068, 27)
df_pred.shape: (3068, 27)


In [27]:
pred_stars = df_pred['star'].tolist()
actual_stars = df_actual['star'].tolist()

print(sorted(list(set(pred_stars))))
print(sorted(list(set(actual_stars))))

[1.0, 2.0, 3.0, 4.0, 5.0]
[1.0, 2.0, 3.0, 4.0, 5.0]


In [28]:
T_correct = 0
T_incorrect = 0

for i, p in enumerate(pred_stars):
    a = actual_stars[i]
    
    if p == a:
        T_correct += 1
    elif p != a:
        T_incorrect += 1
        
print('total correct:', T_correct)
print('total incorrect:', T_incorrect)

print('% correct:', T_correct/(T_correct + T_incorrect))
        

total correct: 2827
total incorrect: 241
% correct: 0.9214471968709257


In [29]:
T_correct = 0
T_incorrect = 0
i_difs = []
for s in [1,2,3,4,5]:
    
    tdf_actual = df_actual[df_actual['star'] == s]
    prvdrs = tdf_actual['PROVIDER_ID'].tolist()
    tdf_pred = df_pred[df_pred['PROVIDER_ID'].isin(prvdrs)]
    
    tdf_pred.sort_values(by=['PROVIDER_ID'], inplace=True)
    tdf_actual.sort_values(by=['PROVIDER_ID'], inplace=True)
    
    pred_stars = tdf_pred['star'].tolist()
    actual_stars = tdf_actual['star'].tolist()
    
    correct = 0
    incorrect = 0
    for i, actual in enumerate(actual_stars):
        try:
            pred = pred_stars[i]

            if actual == pred:
                correct += 1
                T_correct += 1
            else:
                incorrect += 1
                T_incorrect += 1
                i_difs.append(actual - pred)
                #print(actual, '|', pred)
        except:
            continue
            
    print(100*correct/(correct+incorrect), '% correct for ' + str(s) + ' star hospitals')
    print('No. correct:', correct, '| Total:', correct+incorrect, '\n')
    
print(100*T_correct/(T_correct+T_incorrect), '% correct for all hospitals')
print('No. correct:', T_correct, '| Total:', T_correct + T_incorrect, '\n')

print(np.nanmax(i_difs), ',', i_difs.count(np.nanmax(i_difs)))
print(np.nanmin(i_difs), ',', i_difs.count(np.nanmin(i_difs)))
print(np.nanmean(i_difs))
print(np.nanstd(i_difs))

100.0 % correct for 1 star hospitals
No. correct: 250 | Total: 250 

96.84684684684684 % correct for 2 star hospitals
No. correct: 645 | Total: 666 

91.83908045977012 % correct for 3 star hospitals
No. correct: 799 | Total: 870 

90.13732833957553 % correct for 4 star hospitals
No. correct: 722 | Total: 801 

85.44698544698545 % correct for 5 star hospitals
No. correct: 411 | Total: 481 

92.14471968709258 % correct for all hospitals
No. correct: 2827 | Total: 3068 

2.0 , 1
-1.0 , 29
0.7634854771784232
0.6553915310085403


In [30]:
def get_smape(obs, exp, label=None):
    if len(obs) != len(exp):
        raise ValueError("Both lists must have the same length")

    if min(obs) < 0 or min(exp) < 0:
        if min(obs) < 0:
            obs = -1 * np.array(obs)
            obs = obs.tolist()
        if min(exp) < 0:
            exp = -1 * np.array(exp)
            exp = exp.tolist()

    n = len(obs)
    smape_values = []

    for i in range(n):
        if np.isnan(obs[i]) or np.isnan(exp[i]):
            continue
                                        
        numerator = 2 * np.abs(obs[i] - exp[i])
        denominator = np.abs(obs[i]) + np.abs(exp[i])

        # Avoid division by zero
        if denominator == 0:
            smape = 0  # Both obs[i] and exp[i] are zero, no error
        else:
            smape = (numerator / denominator) * 100

        smape_values.append(smape)

    average_smape = sum(smape_values) / n
    return average_smape

In [31]:
print(df_actual.shape)
df_actual.head()

(3068, 27)


,PROVIDER_ID,Std_Outcomes_Mortality_score,Std_Outcomes_Readmission_score,Std_Outcomes_Safety_score,Std_PatientExp_score,Std_Process_score,std_weight_PatientExperience,std_weight_Readmission,std_weight_Mortality,std_weight_safety,std_weight_Process,weight_PatientExperience,weight_Outcomes_Readmission,weight_Outcomes_Mortality,weight_Outcomes_Safety,weight_Process,summary_score,Outcomes_Mortality_cnt,Outcomes_safety_cnt,Outcomes_Readmission_cnt,Patient_Experience_cnt,Process_cnt,Total_measure_group_cnt,MortSafe_Group_cnt,report_indicator,cnt_grp,star
0,010001,-0.647058,0.284081,0.312074,-0.127836,-1.024044,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.162208,7,8,11,8,10,5,2,1,3) # of groups=5,3.0
1,010005,-1.564103,0.560369,-0.237844,-0.166838,-0.302742,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.346181,6,7,10,8,11,5,2,1,3) # of groups=5,2.0
2,010006,-1.694318,-0.554988,-0.089526,-1.241108,-0.179350,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.809109,7,7,9,8,11,5,2,1,3) # of groups=5,1.0
3,010007,-2.407150,-0.488553,0.022657,0.993806,-0.659760,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.492604,3,2,6,8,7,4,1,1,2) # of groups=4,2.0
5,010011,-0.517349,-0.624302,0.428770,0.134223,-2.385055,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.413511,7,7,9,8,8,5,2,1,3) # of groups=5,2.0


In [32]:
print(df_pred.shape)
df_pred.head()

(3068, 27)


,PROVIDER_ID,Std_Outcomes_Mortality_score,Std_Outcomes_Readmission_score,Std_Outcomes_Safety_score,Std_PatientExp_score,Std_Process_score,std_weight_PatientExperience,std_weight_Readmission,std_weight_Mortality,std_weight_safety,std_weight_Process,weight_PatientExperience,weight_Outcomes_Readmission,weight_Outcomes_Mortality,weight_Outcomes_Safety,weight_Process,summary_score,Outcomes_Mortality_cnt,Outcomes_safety_cnt,Outcomes_Readmission_cnt,Patient_Experience_cnt,Process_cnt,Total_measure_group_cnt,MortSafe_Group_cnt,report_indicator,cnt_grp,star
0,010001,-0.648411,0.284587,0.310804,-0.138318,-1.039587,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.166845,7,8,11,8,10,5,2,1,3) # of groups=5,3.0
1,010005,-1.566438,0.560509,-0.228638,-0.176647,-0.315833,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.348367,6,7,10,8,11,5,2,1,3) # of groups=5,2.0
2,010006,-1.696213,-0.554222,-0.087562,-1.248599,-0.188854,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.811713,7,7,9,8,11,5,2,1,3) # of groups=5,1.0
3,010007,-2.409810,-0.487315,0.027365,0.981590,-0.655021,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.494000,3,2,6,8,7,4,1,1,2) # of groups=4,2.0
5,010011,-0.518784,-0.622974,0.426891,0.123447,-2.399212,0.22,0.22,0.22,0.22,0.12,0.22,0.22,0.22,0.22,0.12,-0.418018,7,7,9,8,8,5,2,1,3) # of groups=5,2.0


In [33]:

labels = ['summary_score', 'Std_Outcomes_Mortality_score',
          'Std_Outcomes_Readmission_score', 'Std_Outcomes_Safety_score', 
          'Std_PatientExp_score', 'Std_Process_score',
         ]

for label in labels:
    for s in [1,2,3,4,5]:
        
        tdf_actual = df_actual[df_actual['star'] == s]
        prvdrs = tdf_actual['PROVIDER_ID'].tolist()
        tdf_pred = df_pred[df_pred['PROVIDER_ID'].isin(prvdrs)]
        
        tdf_pred.sort_values(by=['PROVIDER_ID'], inplace=True)
        tdf_actual.sort_values(by=['PROVIDER_ID'], inplace=True)
    
        n_perfect = 0
        n_miss = 0
        n_tot = 0

        obs = tdf_actual[label].astype('float').tolist()
        pred = tdf_pred[label].astype('float').tolist()

        result = get_smape(obs, pred, label)
        apd_ls.append(result)

        print(label, ',', s, '| SMAPE:', round(result, 4))
    


summary_score , 1 | SMAPE: 0.7532
summary_score , 2 | SMAPE: 1.1025
summary_score , 3 | SMAPE: 11.9287
summary_score , 4 | SMAPE: 4.4442
summary_score , 5 | SMAPE: 2.1132
Std_Outcomes_Mortality_score , 1 | SMAPE: 1.1204
Std_Outcomes_Mortality_score , 2 | SMAPE: 0.8213
Std_Outcomes_Mortality_score , 3 | SMAPE: 1.0383
Std_Outcomes_Mortality_score , 4 | SMAPE: 1.0514
Std_Outcomes_Mortality_score , 5 | SMAPE: 0.5495
Std_Outcomes_Readmission_score , 1 | SMAPE: 0.2725
Std_Outcomes_Readmission_score , 2 | SMAPE: 0.3912
Std_Outcomes_Readmission_score , 3 | SMAPE: 0.6849
Std_Outcomes_Readmission_score , 4 | SMAPE: 0.517
Std_Outcomes_Readmission_score , 5 | SMAPE: 0.6193
Std_Outcomes_Safety_score , 1 | SMAPE: 3.88
Std_Outcomes_Safety_score , 2 | SMAPE: 3.7214
Std_Outcomes_Safety_score , 3 | SMAPE: 3.9277
Std_Outcomes_Safety_score , 4 | SMAPE: 3.0773
Std_Outcomes_Safety_score , 5 | SMAPE: 2.3268
Std_PatientExp_score , 1 | SMAPE: 1.3869
Std_PatientExp_score , 2 | SMAPE: 3.2802
Std_PatientExp_score